# Querying the *flight phase files*
We introduce here some new methods to select the flight segments, querying for different criteria.

## Data access
* To analyse the data they first have to be loaded by importing the (AC)³airborne meta data catalogue. To do so the ac3airborne package has to be installed. More information on how to do that and about the catalog can be found [here](https://github.com/igmk/ac3airborne-intake#ac3airborne-intake-catalogue).

## Get data

In [1]:
import ac3airborne

In [2]:
cat = ac3airborne.get_intake_catalog()

datasets = []
for campaign in ['ACLOUD', 'AFLUX', 'MOSAiC-ACA']:
    datasets.extend(list(cat[campaign]['P5']['GPS_INS']))
for campaign in ['ACLOUD', 'PAMARCMiP']:
    datasets.extend(list(cat[campaign]['P6']['GPS_INS']))
datasets

['ACLOUD_P5_RF04',
 'ACLOUD_P5_RF05',
 'ACLOUD_P5_RF06',
 'ACLOUD_P5_RF07',
 'ACLOUD_P5_RF08',
 'ACLOUD_P5_RF10',
 'ACLOUD_P5_RF11',
 'ACLOUD_P5_RF13',
 'ACLOUD_P5_RF14',
 'ACLOUD_P5_RF15',
 'ACLOUD_P5_RF16',
 'ACLOUD_P5_RF17',
 'ACLOUD_P5_RF18',
 'ACLOUD_P5_RF19',
 'ACLOUD_P5_RF20',
 'ACLOUD_P5_RF21',
 'ACLOUD_P5_RF22',
 'ACLOUD_P5_RF23',
 'ACLOUD_P5_RF25',
 'AFLUX_P5_RF02',
 'AFLUX_P5_RF03',
 'AFLUX_P5_RF04',
 'AFLUX_P5_RF05',
 'AFLUX_P5_RF06',
 'AFLUX_P5_RF07',
 'AFLUX_P5_RF08',
 'AFLUX_P5_RF09',
 'AFLUX_P5_RF10',
 'AFLUX_P5_RF11',
 'AFLUX_P5_RF12',
 'AFLUX_P5_RF13',
 'AFLUX_P5_RF14',
 'AFLUX_P5_RF15',
 'MOSAiC-ACA_P5_RF02',
 'MOSAiC-ACA_P5_RF03',
 'MOSAiC-ACA_P5_RF04',
 'MOSAiC-ACA_P5_RF05',
 'MOSAiC-ACA_P5_RF06',
 'MOSAiC-ACA_P5_RF07',
 'MOSAiC-ACA_P5_RF08',
 'MOSAiC-ACA_P5_RF09',
 'MOSAiC-ACA_P5_RF10',
 'MOSAiC-ACA_P5_RF11',
 'ACLOUD_P6_RF07',
 'ACLOUD_P6_RF08',
 'ACLOUD_P6_RF09',
 'ACLOUD_P6_RF10',
 'ACLOUD_P6_RF11',
 'ACLOUD_P6_RF12',
 'ACLOUD_P6_RF13',
 'ACLOUD_P6_RF14',
 'ACL

```{note}
Have a look at the attributes of the xarray dataset `ds_gps` for all relevant information on the dataset, such as author, contact, or citation information.
```

In this example we want to look at `ACLOUD_P5_RF14`. First we read the GPS information:

In [3]:
ds_gps = cat['ACLOUD']['P5']['GPS_INS']['ACLOUD_P5_RF14'].to_dask()
ds_gps

ValueError: found the following matches with the input file in xarray's IO backends: ['h5netcdf']. But their dependencies may not be installed, see:
http://xarray.pydata.org/en/stable/user-guide/io.html 
http://xarray.pydata.org/en/stable/getting-started-guide/installing.html

## Load Polar 5 flight phase information
Polar 5 flights are divided into segments to easily access start and end times of flight patterns. For more information have a look at the respective [github](https://github.com/igmk/flight-phase-separation) repository.

At first we want to load the flight segments of (AC)³airborne

In [4]:
meta = ac3airborne.get_flight_segments() 

In order to simplify things we can import the module `flightphase` from the `ac3airborne.tools`.

In [5]:
from ac3airborne.tools import flightphase

IndentationError: unindent does not match any outer indentation level (flightphase.py, line 133)

The next step is to select the flight segments of our flight:

In [ ]:
flight = meta['ACLOUD']['P5']['ACLOUD_P5_RF14']

In [ ]:
flight_query = flightphase.FlightPhaseFile(flight)

## Plots
First, the entire flight is plotted:

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
plt.style.use("../mplstyle/book")

In [ ]:
proj = ccrs.NorthPolarStereo()
extent = (-5.0, 24.0, 78.0, 83.0)

fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection=proj)
ax.set_extent(extent)

ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.LAND)
ax.gridlines()
ax.coastlines()

nya_lat = 78.924444
nya_lon = 11.928611

ax.plot(nya_lon, nya_lat, 'ro', transform=ccrs.PlateCarree())
ax.text(nya_lon, nya_lat+0.05, 'Ny-Ålesund', transform=ccrs.PlateCarree())

line_all = ax.plot(ds_gps.lon, ds_gps.lat, transform=ccrs.PlateCarree())

plt.show()

### Query all segments of a specific kind

As an example only `high_level` flights are plotted in the following (only one kind of pattern):

In [ ]:
queried = flight_query.selectKind(['high_level'])
queried

Plot the selected segments:

In [ ]:
ll = line_all.pop(0)
ll.remove()

lines = []
for q in queried:
    start = q['start']
    end   = q['end']
    
    line = ax.plot(ds_gps.lon.sel(time=slice(start, end)),
                   ds_gps.lat.sel(time=slice(start, end)),
                   transform=ccrs.PlateCarree())
    lines.append(line)
    
fig

 Of course, also more than one kind of pattern can be selected:

In [ ]:
kinds = ['a-train_underflight', 'nya_overflight', 'polarstern_overflight']

queried = flight_query.selectKind(kinds)
queried

Plot the selected segments:

In [ ]:
for l in lines:
    ll = l.pop(0)
    ll.remove()
    
lines = []
for q in queried:
    start = q['start']
    end   = q['end']
    
    line = ax.plot(ds_gps.lon.sel(time=slice(start, end)),
                   ds_gps.lat.sel(time=slice(start, end)),
                   transform=ccrs.PlateCarree())
    lines.append(line)
    
fig

### Sorting out all segments given a list of kinds

In [ ]:
sort_out = ['short_turn', 'holding_pattern', 'major_ascent', 'major_descent', 'high_level']

queried = flight_query.selectKind(sort_out, invertSelection=True)
queried

Plot the selected segments:

In [ ]:
for l in lines:
    ll = l.pop(0)
    ll.remove()
    
lines = []
for q in queried:
    start = q['start']
    end   = q['end']
    
    line = ax.plot(ds_gps.lon.sel(time=slice(start, end)),
                   ds_gps.lat.sel(time=slice(start, end)),
                   transform=ccrs.PlateCarree())
    lines.append(line)
    
fig

### Query a segment with a specific value in a specific attribute

#### Example 1: Select element with specific name

In [ ]:
attribute = 'name'
value = 'racetrack pattern 1'

queried = flight_query.select(attribute, value) 
queried

Plot the selected elements:

In [ ]:
for l in lines:
    ll = l.pop(0)
    ll.remove()
    
lines = []
for q in queried:
    start = q['start']
    end   = q['end']
    
    line = ax.plot(ds_gps.lon.sel(time=slice(start, end)),
                   ds_gps.lat.sel(time=slice(start, end)),
                   transform=ccrs.PlateCarree())
    lines.append(line)
    
fig

Also with inverted selection:

In [ ]:
queried = flight_query.select(attribute, value, invertSelection=True) 
queried

Plot the selected elements:

In [ ]:
for l in lines:
    ll = l.pop(0)
    ll.remove()
    
lines = []
for q in queried:
    start = q['start']
    end   = q['end']
    
    line = ax.plot(ds_gps.lon.sel(time=slice(start, end)),
                   ds_gps.lat.sel(time=slice(start, end)),
                   transform=ccrs.PlateCarree())
    lines.append(line)
    
fig

#### Example 2: Specific level

In [ ]:
attribute = 'levels'
value = 9800

queried = flight_query.select(attribute, value)
queried

Plot the selected elements:

In [ ]:
for l in lines:
    ll = l.pop(0)
    ll.remove()
    
lines = []
for q in queried:
    start = q['start']
    end   = q['end']
    
    line = ax.plot(ds_gps.lon.sel(time=slice(start, end)),
                   ds_gps.lat.sel(time=slice(start, end)),
                   transform=ccrs.PlateCarree())
    lines.append(line)
    
fig

But what if we want only a specific level without ascent and descent? We can require that the attribute value is strictly equal to the one, that we give, i.e. there are no other values:

In [ ]:
queried = flight_query.select(attribute, value, strict=True)
queried

This functions only if the attribute has more values, as for level or kind. Now plot the selected elements:

In [ ]:
for l in lines:
    ll = l.pop(0)
    ll.remove()
    
lines = []
for q in queried:
    start = q['start']
    end   = q['end']
    
    line = ax.plot(ds_gps.lon.sel(time=slice(start, end)),
                   ds_gps.lat.sel(time=slice(start, end)),
                   transform=ccrs.PlateCarree())
    lines.append(line)
    
fig

## Warnings

In the previous cells warnings were intentionally suppressed. But the flightphase tools normally also show a warning if some of the selected segments contain so-called "irregularities". So we first reactivate the warnings:

In [ ]:
warnings.simplefilter("always")

We repeat one of the previous examples in order to show how the warnings of the flightphase tools look like:

In [ ]:
kinds = ['a-train_underflight', 'nya_overflight', 'polarstern_overflight']

queried = flight_query.selectKind(kinds)
queried

The first part of the warning tells us which segment is generating the warning. In this case we have two warnings generated by the segments `ACLOUD_P5_RF14_ma` and `ACLOUD_P5_RF14_ld01`. The second part is the value of the attribute `irregularities`.  A warning is generated when this attribute is not empty. You can access the content of this attribute even if the warnings are suppressed, as you can see in the examples above.

## Select a segment at a specific time or within a specific time range

If you want to select all the segments within a specific time range you can do it with the function `findSegments` which takes as parameters the start time and the end time in the `%Y-%m-%d %H:%M:%S` format.

In [ ]:
starttime = '2017-06-08 12:15:00'
endtime   = '2017-06-08 12:45:00'

queried = flight_query.findSegments(starttime, endtime)
queried

For a single time point you just take the end time to be the same as the start time

In [ ]:
starttime = '2017-06-08 12:00:00'
endtime   = starttime

queried = flight_query.findSegments(starttime, endtime)
queried